In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date
from scipy import stats
import statsmodels.api as sm
from itertools import product
import seaborn as sns
import warnings
import pickle

In [2]:
df = pd.read_csv("coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv",delimiter= ',')

In [3]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit = 's')
df.set_index('Timestamp', inplace=True)
df = df.resample('D').mean()

In [4]:
df1 = df.iloc[:-30]

In [5]:
Q = range(0,2)
q = range(0,2)
P = range(0,2)
p = range(0,2)
D = 1
d = 1
parameters = product(p,q, P, Q)
parameters_list = list(parameters)
len(parameters_list)

res = []
ai = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
  try:
    model = sm.tsa.statespace.SARIMAX(df1['Weighted_Price'], order = (param[0], d, param[1]),
                                      seasonal_order = (param[2], D, param[3], 12)).fit(disp = -1)
  except ValueError:
    continue
  aic = model.aic
  if aic < ai:
    ml = model
    ai = aic
    para = param
  res.append([param, model.aic])

In [6]:
filename = 'Sarima.pickle'
pickle.dump(ml, open(filename, 'ab'))